In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# From Blips to Images 🖼️

## 🎯 Learning goals

- **Understand** how range–angle scans form a 2D radar image by using **pulses and beam steering.**
- **Compare** stationary beam scanning vs. **moving-platform imaging**.
- **Explore trade-offs**: pulse length for range resolution and beamwidth for angle.

# Radar Imaging with a Real Aperture Array 📡

Believe it or not, but you’ve already learned almost **all the key concepts** needed to start building radar images! 🚀 Let’s imagine we have an **array of antenna elements** that can transmit short **pulsed signals**. Our goal: use it to make a radar image of the area in front of us. But… how? 🤔  

---

## 🎛 Steering the Beam

With an array, we can **steer the beam** simply by adjusting the **phases** of the transmitted signals. 🎯  
- To **transmit** energy in a certain direction, we give each antenna element a tiny phase offset so their waves add up in that direction 🌊➕🌊.  
- To **receive** energy from that direction, we apply the same phase trick to the incoming signals, so they align perfectly before we sum them.  
- Result: the signals combine **coherently** and we can focus in any direction without moving the antenna! 🔍

---

## 🗺 From Pulses to a 2D Image

Here’s the trick:  
1. We send a **short pulse** and measure the **time delay** of echoes from objects in the beam. This gives us **range** (distance). 📏  
2. We **scan the beam** over a range of angles in front of the radar, i.e. we steer the beam **both on transmit and receive** to illuminate and capture echoes from a certain direction. 📐  
3. We record each echo line (range profile) for each beam angle and stack them together. 💾  

The result? A 2D **range–angle map** of reflected radar energy. That's what we call **a radar “image”** of the scene in front of us! 🖼

---

## ⚡ A Different Kind of Image

Let's spend a moment to think what is a bit unsual here:
- The **image grid** is on the **ground plane** (like a top-down map) 📍.  
- But we didn’t measure it from above, we measured it **from the side**!  
- Optical cameras form images **perpendicular** to incoming light rays, but radars form images using **time delay** (range) and **beam direction** (azimuth angle).  

This makes radar images **quirky** for anyone used to optical photos, as objects may appear distorted in ways unique to radar. We’ll explore these quirks in more detail later. 😉

---

## 🎮 Try It Yourself!

In the simulation below, our radar is scanning two small reflective targets (think shiny metal spheres) at fixed locations. The radar and the targets are both located on the ground plane in this example. You can:
- Adjust the target positions 🖱  
- Change the **beamwidth** and **pulse duration** of the radar ⚙  
- Watch how the measured **range–angle map** changes, and how the radar “image” forms as the scan progresses! 🔦

In [ ]:
def build_radar_scan_sim():
    """
    Voila-friendly radar scan demo without matplotlib.animation.
    - Time is driven by ipywidgets Play + IntSlider.
    - Sliders: target 1/2 (x,y), beamwidth (deg), pulse duration (s).
    - 'Auto-update' toggle and 'Render' button to avoid expensive redraws when desired.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import (
        FloatSlider, FloatLogSlider, IntSlider, Play, jslink,
        ToggleButton, Button, HBox, VBox, Layout, Output, Label, Dropdown
    )
    from IPython.display import display, clear_output
    import os
    from datetime import datetime

    # -----------------------
    # Constants & defaults
    # -----------------------
    c = 299_792_458.0  # m/s

    # Geometry / sampling (kept close to your original)
    wavelength = 10.0           # m
    beamwidth_default = 30.0    # deg
    pulse_default = 5e-8        # s
    grid_size = 100
    space_extent = 20.0 * wavelength

    # Animation structure: wave_frames per radar position
    wave_frames = 20
    num_positions = 40
    total_frames = wave_frames * num_positions
    max_scan_angle_deg = 20.0

    # Precompute azimuth scan (radians) and also degrees for labeling
    azimuth_scan_angles = np.radians(
        np.linspace(-max_scan_angle_deg, max_scan_angle_deg, num_positions)
    )
    azimuth_deg = np.degrees(azimuth_scan_angles)

    # Pulse window: time for pulse to go out & back across plotting extent
    pulse_time = 4.0 * space_extent / c

    # Fixed grid (top-to-bottom y for imshow consistency)
    x = np.linspace(-space_extent, space_extent, grid_size)
    y = np.linspace(0.0, 2.0 * space_extent, grid_size)
    X, Y = np.meshgrid(x, y)
    extent_cart = (-space_extent, space_extent, 2.0 * space_extent, 0.0)

    # Radar path (keep x=0 as in original; change to linspace to move along-track)
    radar_x_positions = np.zeros((num_positions,), dtype=float)
    radar_y = 0.0

    # Range-time sampling for 1D echoes (same heuristic as original)
    def make_range_axis(pulse_duration):
        fs = 10.0 / pulse_duration
        n_bins = int(max(1, pulse_time * fs))
        t = np.linspace(0.0, pulse_time, n_bins)
        rng = c * t / 2.0
        return t, rng

    # -----------------------
    # Widgets
    # -----------------------
    t1x = FloatSlider(description="T1 x (m)", min=-space_extent, max=space_extent, step=1.0,
                      value=30.0, continuous_update=False, readout_format=".1f",
                      layout=Layout(width="48%"))
    t1x.style.description_width = 'initial'

    t1y = FloatSlider(description="T1 y (m)", min=0.0, max=2.0 * space_extent, step=1.0,
                      value=200.0, continuous_update=False, readout_format=".1f",
                      layout=Layout(width="48%"))
    t1y.style.description_width = 'initial'

    t2x = FloatSlider(description="T2 x (m)", min=-space_extent, max=space_extent, step=1.0,
                      value=-60.0, continuous_update=False, readout_format=".1f",
                      layout=Layout(width="48%"))
    t2x.style.description_width = 'initial'

    t2y = FloatSlider(description="T2 y (m)", min=0.0, max=2.0 * space_extent, step=1.0,
                      value=250.0, continuous_update=False, readout_format=".1f",
                      layout=Layout(width="48%"))
    t2y.style.description_width = 'initial'

    beam = FloatSlider(description="Beamwidth (°)", min=1.0, max=90.0, step=0.5,
                       value=beamwidth_default, continuous_update=False, readout_format=".1f",
                       layout=Layout(width="48%"))
    beam.style.description_width = 'initial'

    pulse = FloatLogSlider(description="Pulse (s)", base=10, min=-8, max=-6, step=0.1,
                           value=pulse_default, continuous_update=False, readout_format=".2e",
                           layout=Layout(width="48%"))
    pulse.style.description_width = 'initial'

    play = Play(value=0, min=0, max=total_frames - 1, step=1, interval=100)
    t_slider = IntSlider(value=0, min=0, max=total_frames - 1, step=1, description="Time")
    jslink((play, 'value'), (t_slider, 'value'))

    auto_update = ToggleButton(value=True, description="Auto-update", icon="bolt")
    render_btn = Button(description="Render", button_style="primary", icon="refresh")
    reset_btn = Button(description="Reset")
    save_btn  = Button(description="Save PNG")
    
    cmap_dd = Dropdown(options=['jet', 'viridis', 'plasma', 'gray', 'coolwarm'], 
                       value='gray', description='Colormap:', 
                       style={'description_width':'initial'})

    readout = Label()
    controls1 = HBox([t1x, t1y])
    controls2 = HBox([t2x, t2y])
    controls3 = HBox([beam, pulse, cmap_dd])
    controls4 = HBox([play, t_slider, auto_update, render_btn, reset_btn, save_btn],
                     layout=Layout(align_items='center', column_gap='10px'))

    # -----------------------
    # Figure / artists
    # -----------------------
    out = Output()
    fig, (ax_wave, ax_sar) = plt.subplots(1, 2, figsize=(12, 6))
    plt.close(fig)  # prevent static capture by the notebook

    # Left: pulse propagation & targets
    im_wave = ax_wave.imshow(
        np.zeros_like(X), extent=extent_cart, cmap=cmap_dd.value,
        vmin=0, vmax=1.0, origin='upper', animated=False
    )
    scatter_tg = ax_wave.scatter([], [], color="red", s=100, marker="x", label="Target")
    # sc_radar = ax_wave.scatter([], [], color="white", s=50, marker='x')
    antenna_size = wavelength / np.radians(beamwidth_default)
    nb_elements = int(antenna_size / (wavelength/2))
    element_x = np.linspace(-antenna_size/2, antenna_size/2, nb_elements)
    element_y = np.zeros_like(element_x)
    (line_aperture,) = ax_wave.plot(element_x, element_y, color='white', linestyle='-', marker='x', linewidth=4, label="Antenna Array")
    ax_wave.set_title("Radar Data Collection")
    ax_wave.set_xlabel("X Position (m)")
    ax_wave.set_ylabel("Y Position (m)")
    ax_wave.legend(loc='upper right')

    # Right: SAR echo data heatmap (range vs azimuth angle in degrees)
    # initialize with correct extent (range on x, azimuth deg on y; origin upper => y descends)
    im_sar = None
    sar_cbar = None
    def sar_extent(range_axis):
        return [range_axis[0], range_axis[-1], azimuth_deg[-1], azimuth_deg[0]]

    # -----------------------
    # Helpers
    # -----------------------
    def antenna_pattern(theta_rad, beam_deg):
        # Simplified sinc^2; theta in radians, beamwidth converted to radians here
        return np.sinc(theta_rad / np.radians(beam_deg)) ** 2

    def rect(tvals, width=1.0, center=0.0):
        return np.where(
            (tvals >= center - width/2.0) & (tvals <= center + width/2.0), 1.0, 0.0
        )

    def compute_row_echo(position_idx, beam_deg, pulse_duration, targets, t_axis):
        """Compute the 1D echo row for a given radar position (end-of-pulse)."""
        radar_x = radar_x_positions[position_idx]
        radar_y_loc = radar_y
        scan_angle = azimuth_scan_angles[position_idx]
        row = 0.0
        for tx, ty, ta in targets:
            r_dist = np.hypot(radar_x - tx, radar_y_loc - ty)
            targ_az = np.arctan2(tx - radar_x, ty - radar_y_loc)
            ant_amp = antenna_pattern(targ_az - scan_angle, beam_deg)
            return_time = 2.0 * r_dist / c
            row += ant_amp * ta * rect(t_axis, width=pulse_duration, center=return_time)
        return row

    # -----------------------
    # Draw routine
    # -----------------------
    def draw(frame_idx):
        nonlocal im_sar, sar_cbar

        # Parameters from UI
        beam_deg = float(beam.value)
        pulse_s = float(pulse.value)
        targets = [
            (float(t1x.value), float(t1y.value), 0.2),
            (float(t2x.value), float(t2y.value), 0.15),
        ]

        # Time bookkeeping
        pos_idx = int(frame_idx // wave_frames)
        time_idx = int(frame_idx % wave_frames)
        frac = (time_idx / max(1, (wave_frames - 1)))  # [0..1]
        current_time = frac * pulse_time

        # Geometry fields
        radar_x = radar_x_positions[pos_idx]
        R_arr = np.hypot(X - radar_x, Y - radar_y)
        theta = np.arctan2(X, Y)                     # radians
        scan_angle = azimuth_scan_angles[pos_idx]    # radians
        az_angle = theta - scan_angle                # radians

        # Outgoing pulse ring
        pulse_distance = current_time * c
        pulse_width = pulse_s * c
        Z_total = np.zeros_like(R_arr)
        in_pulse_out = (R_arr > pulse_distance - pulse_width / 2.0) & (R_arr < pulse_distance + pulse_width / 2.0)
        if np.any(in_pulse_out):
            Z_total[in_pulse_out] = 1.0

        # Apply antenna pattern
        Z_total *= antenna_pattern(az_angle, beam_deg)

        # Target reflections (visual ring)
        for tx, ty, ta in targets:
            r_tx = np.hypot(X - tx, Y - ty)
            r_radar_to_target = np.hypot(radar_x - tx, radar_y - ty)
            targ_az = np.arctan2(tx - radar_x, ty - radar_y)
            ant_amp = antenna_pattern(targ_az - scan_angle, beam_deg)

            if pulse_distance >= r_radar_to_target:
                leftover = pulse_distance - r_radar_to_target
                in_pulse_t = (r_tx > leftover - pulse_width/2.0) & (r_tx < leftover + pulse_width/2.0)
                if np.any(in_pulse_t):
                    Z_total[in_pulse_t] += ant_amp * ta

        # Update left image & artists
        im_wave.set_data(Z_total)
        # sc_radar.set_offsets(np.c_[[radar_x], [radar_y]])

        # Update targets scatter
        scatter_tg.set_offsets(np.c_[[t[0] for t in targets], [t[1] for t in targets]])

        # Build/update SAR heatmap (recompute rows up to last *completed* position)
        t_axis, range_axis = make_range_axis(pulse_s)
        echo_data = np.zeros((num_positions, t_axis.size), dtype=float)
        for p in range(min(pos_idx, num_positions)):  # exclude current in-progress row
            echo_data[p, :] = compute_row_echo(p, beam_deg, pulse_s, targets, t_axis)

        if im_sar is None:
            im_sar = ax_sar.imshow(
                echo_data, aspect='auto', cmap=cmap_dd.value,
                extent=sar_extent(range_axis), origin='upper'
            )
            ax_sar.set_title("Radar Echo Data")
            ax_sar.set_xlabel("Range (m)")
            ax_sar.set_ylabel("Azimuth angle (deg)")
            sar_cbar = fig.colorbar(im_sar, ax=ax_sar, label='Echo Amplitude')  # <-- no plt.*
        else:
            im_sar.set_data(echo_data)
            im_sar.set_extent(sar_extent(range_axis))

        # Keep color scale consistent
        im_sar.set_clim(0, sum([t[2] for t in targets]))

        antenna_size = wavelength / np.radians(beam_deg)
        nb_elements = int(antenna_size / (wavelength/2))
        element_x = np.linspace(-antenna_size/2, antenna_size/2, nb_elements)
        element_y = np.zeros_like(element_x)
        line_aperture.set_data(element_x, element_y)

        # Readout text
        readout.value = (
            f"pos={pos_idx+1}/{num_positions}, frame={frame_idx+1}/{total_frames}, scan angle={np.degrees(scan_angle):.1f}° | "
            f"beam={beam_deg:.1f}°, pulse={pulse_s:.2e}s | "
            f"T1=({targets[0][0]:.1f},{targets[0][1]:.1f}), "
            f"T2=({targets[1][0]:.1f},{targets[1][1]:.1f})"
        )

        # Draw figure into output
        with out:
            clear_output(wait=True)
            display(fig)

    # -----------------------
    # Wiring
    # -----------------------
    def maybe_draw(change=None):
        if auto_update.value:
            draw(t_slider.value)

    def _ensure_outputs_dir():
        out_dir = os.path.join('outputs')
        os.makedirs(out_dir, exist_ok=True)
        return out_dir

    def on_reset(_):
        t1x.value, t1y.value = 30.0, 200.0
        t2x.value, t2y.value = -60.0, 250.0
        beam.value = beamwidth_default
        pulse.value = pulse_default
        t_slider.value = 0
        draw(t_slider.value)

    def on_save(_):
        out_dir = _ensure_outputs_dir()
        ts = datetime.now().strftime('%Y%m%d_%H%M%S')
        base = f'radar_scan_{ts}'
        fig.savefig(os.path.join(out_dir, base + '.png'), dpi=120, bbox_inches='tight')

    # Time controls
    t_slider.observe(lambda ch: draw(ch["new"]), names='value')

    # Param controls
    for w in (t1x, t1y, t2x, t2y, beam, pulse):
        w.observe(maybe_draw, names='value')
    
    def _update_colormap(_):
        im_wave.set_cmap(cmap_dd.value)
        if im_sar is not None:
            im_sar.set_cmap(cmap_dd.value)
        if auto_update.value:
            draw(t_slider.value)
    cmap_dd.observe(_update_colormap, names='value')

    # Manual render button
    render_btn.on_click(lambda _: draw(t_slider.value))
    reset_btn.on_click(on_reset)
    save_btn.on_click(on_save)

    # Initial render
    draw(t_slider.value)

    return VBox([controls1, controls2, controls3, controls4, readout, out])

# Build and show (safe alongside other sims / Voila):
radar_app = build_radar_scan_sim()
display(radar_app)

### 🧪 Try this
- Start with the defaults; scrub Time and watch the pulse ring and echo map grow.
- Narrow the **Beamwidth**; observe a thinner illuminated slice and sharper azimuth detail.
- Shorten the **Pulse**; see improved range separation.
- Move targets to same range but different x (azimuth); compare angular separability.

## ✈️ Side-Looking Airborne Radar (SLAR)  

Let’s explore another way to use a **real antenna array**, also known as a **real aperture radar**, to form an image of the ground. This approach works beautifully from both airborne and spaceborne platforms. ✈️ 🛰️ 

In the **beam scanning** example earlier, the radar stayed in one place and we steered the beam electronically. But there’s another option: we can “scan” the scene simply by **moving the radar itself**. This happens naturally when the radar is mounted on a **moving platform** such as an airplane or a satellite. This will work even if we don't have an array antenna, but for example a reflector 📡.

Later, with **synthetic aperture radar (SAR)**, we’ll use this motion in an even smarter way. But let’s not get ahead of ourselves. 🤓  

---

### 📡 How SLAR Works  

Imagine the radar platform flying along a straight path, with the antenna pointed **exactly sideways** ➡️, perpendicular to the flight direction. As the platform moves forward, we:  

1️⃣ **Transmit** a pulse ⚡  
2️⃣ **Receive** the echoes from the ground to the side 🌍🔊  
3️⃣ **Move forward** and repeat ✈️🔁  

Every time we transmit and receive, we store one **line of echo data** 📝, just as in the scanning example. Each line corresponds to echoes from one position along the flight path (the **azimuth** direction).  

Now, if we **stack** these lines of data one after another 📚—each new row representing the echoes from the next position—we get a **2D image of the ground**! 🗺️✨  

---

### 🔍 Why It Works  

In SLAR, the scanning of the scene happens simply because the **platform moves**, sweeping the beam over different parts of the ground. That's why we don't have to direct the beam to illuminate and receive echoes from different parts of the ground. 🔦  The **azimuth resolution** we get here is exactly the same as in our earlier beam-scanning example, as it’s limited by the **beamwidth** of the used antenna. More on this in just a moment!

In [ ]:
def build_slar_sim():
    """
    Voila-friendly radar scan demo without matplotlib.animation.
    - Time is driven by ipywidgets Play + IntSlider.
    - Sliders: target 1/2 (x,y), beamwidth (deg), pulse duration (s).
    - 'Auto-update' toggle and 'Render' button to avoid expensive redraws when desired.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import (
        FloatSlider, FloatLogSlider, IntSlider, Play, jslink,
        ToggleButton, Button, HBox, VBox, Layout, Output, Label, Dropdown
    )
    from IPython.display import display, clear_output
    import os
    from datetime import datetime

    # -----------------------
    # Constants & defaults
    # -----------------------
    c = 299_792_458.0  # m/s

    # Geometry / sampling (kept close to your original)
    wavelength = 10.0           # m
    beamwidth_default = 30.0    # deg
    pulse_default = 5e-8        # s
    grid_size = 100
    space_extent = 20.0 * wavelength

    # Animation structure: wave_frames per radar position
    wave_frames = 20
    num_positions = 40
    total_frames = wave_frames * num_positions
    ap = 20.0

    # Pulse window: time for pulse to go out & back across plotting extent
    pulse_time = 4.0 * space_extent / c

    # Fixed grid (top-to-bottom y for imshow consistency)
    x = np.linspace(-space_extent, space_extent, grid_size)
    y = np.linspace(0.0, 2.0 * space_extent, grid_size)
    X, Y = np.meshgrid(x, y)
    extent_cart = (-space_extent, space_extent, 2.0 * space_extent, 0.0)

    # Radar path (keep x=0 as in original; change to linspace to move along-track)
    radar_x_positions = np.linspace(-0.5 * space_extent, 0.5 * space_extent, num_positions)
    radar_y = 0.0

    # Range-time sampling for 1D echoes (same heuristic as original)
    def make_range_axis(pulse_duration):
        fs = 10.0 / pulse_duration
        n_bins = int(max(1, pulse_time * fs))
        t = np.linspace(0.0, pulse_time, n_bins)
        rng = c * t / 2.0
        return t, rng

    # -----------------------
    # Widgets
    # -----------------------
    t1x = FloatSlider(description="T1 x (m)", min=-space_extent, max=space_extent, step=1.0,
                      value=30.0, continuous_update=False, readout_format=".1f",
                      layout=Layout(width="48%"))
    t1x.style.description_width = 'initial'
    t1y = FloatSlider(description="T1 y (m)", min=0.0, max=2.0 * space_extent, step=1.0,
                      value=200.0, continuous_update=False, readout_format=".1f",
                      layout=Layout(width="48%"))
    t1y.style.description_width = 'initial'
    t2x = FloatSlider(description="T2 x (m)", min=-space_extent, max=space_extent, step=1.0,
                      value=-60.0, continuous_update=False, readout_format=".1f",
                      layout=Layout(width="48%"))
    t2x.style.description_width = 'initial'
    t2y = FloatSlider(description="T2 y (m)", min=0.0, max=2.0 * space_extent, step=1.0,
                      value=250.0, continuous_update=False, readout_format=".1f",
                      layout=Layout(width="48%"))
    t2y.style.description_width = 'initial'

    beam = FloatSlider(description="Beamwidth (°)", min=1.0, max=90.0, step=0.5,
                       value=beamwidth_default, continuous_update=False, readout_format=".1f",
                       layout=Layout(width="48%"))
    beam.style.description_width = 'initial'

    pulse = FloatLogSlider(description="Pulse (s)", base=10, min=-8, max=-6, step=0.1,
                           value=pulse_default, continuous_update=False, readout_format=".2e",
                           layout=Layout(width="48%"))
    pulse.style.description_width = 'initial'

    play = Play(value=0, min=0, max=total_frames - 1, step=1, interval=100)
    t_slider = IntSlider(value=0, min=0, max=total_frames - 1, step=1, description="Time")
    jslink((play, 'value'), (t_slider, 'value'))

    auto_update = ToggleButton(value=True, description="Auto-update", icon="bolt")
    render_btn = Button(description="Render", button_style="primary", icon="refresh")
    reset_btn = Button(description="Reset")
    save_btn  = Button(description="Save PNG")
    
    cmap_dd = Dropdown(options=['jet', 'viridis', 'plasma', 'gray', 'coolwarm'], 
                       value='gray', description='Colormap:', 
                       style={'description_width':'initial'})

    readout = Label()
    controls1 = HBox([t1x, t1y])
    controls2 = HBox([t2x, t2y])
    controls3 = HBox([beam, pulse, cmap_dd])
    controls4 = HBox([play, t_slider, auto_update, render_btn, reset_btn, save_btn],
                     layout=Layout(align_items='center', column_gap='10px'))

    # -----------------------
    # Figure / artists
    # -----------------------
    out = Output()
    fig, (ax_wave, ax_sar) = plt.subplots(1, 2, figsize=(12, 6))
    plt.close(fig)  # prevent static capture by the notebook

    # Left: pulse propagation & targets
    im_wave = ax_wave.imshow(
        np.zeros_like(X), extent=extent_cart, cmap=cmap_dd.value,
        vmin=0, vmax=1.0, origin='upper', animated=False
    )
    scatter_tg = ax_wave.scatter([], [], color="red", s=100, marker="x", label="Target")
    # sc_radar = ax_wave.scatter([], [], color="white", s=50, marker='x')
    antenna_size = wavelength / np.radians(beamwidth_default)
    nb_elements = int(antenna_size / (wavelength/2))
    element_x = np.linspace(-antenna_size/2, antenna_size/2, nb_elements)
    element_y = np.zeros_like(element_x)
    (line_aperture,) = ax_wave.plot(element_x, element_y, color='white', linestyle='-', marker='x', linewidth=4, label="Antenna Array")
    ax_wave.set_title("Radar Data Collection")
    ax_wave.set_xlabel("X Position (m)")
    ax_wave.set_ylabel("Y Position (m)")
    ax_wave.legend(loc='upper right')

    # Right: SAR echo data heatmap (range vs azimuth angle in degrees)
    # initialize with correct extent (range on x, azimuth deg on y; origin upper => y descends)
    im_sar = None
    sar_cbar = None
    def sar_extent(range_axis):
        return [range_axis[0], range_axis[-1], radar_x_positions[-1], radar_x_positions[0]]

    # -----------------------
    # Helpers
    # -----------------------
    def antenna_pattern(theta_rad, beam_deg):
        # Simplified sinc^2; theta in radians, beamwidth converted to radians here
        return np.sinc(theta_rad / np.radians(beam_deg)) ** 2

    def rect(tvals, width=1.0, center=0.0):
        return np.where(
            (tvals >= center - width/2.0) & (tvals <= center + width/2.0), 1.0, 0.0
        )

    def compute_row_echo(position_idx, beam_deg, pulse_duration, targets, t_axis):
        """Compute the 1D echo row for a given radar position (end-of-pulse)."""
        radar_x = radar_x_positions[position_idx]
        radar_y_loc = radar_y
        row = 0.0
        for tx, ty, ta in targets:
            r_dist = np.hypot(radar_x - tx, radar_y_loc - ty)
            targ_az = np.arctan2(tx - radar_x, ty - radar_y_loc)
            ant_amp = antenna_pattern(targ_az, beam_deg)
            return_time = 2.0 * r_dist / c
            row += ant_amp * ta * rect(t_axis, width=pulse_duration, center=return_time)
        return row

    # -----------------------
    # Draw routine
    # -----------------------
    def draw(frame_idx):
        nonlocal im_sar, sar_cbar

        # Parameters from UI
        beam_deg = float(beam.value)
        pulse_s = float(pulse.value)
        targets = [
            (float(t1x.value), float(t1y.value), 0.2),
            (float(t2x.value), float(t2y.value), 0.15),
        ]

        # Time bookkeeping
        pos_idx = int(frame_idx // wave_frames)
        time_idx = int(frame_idx % wave_frames)
        frac = (time_idx / max(1, (wave_frames - 1)))  # [0..1]
        current_time = frac * pulse_time

        # Geometry fields
        radar_x = radar_x_positions[pos_idx]
        R_arr = np.hypot(X - radar_x, Y - radar_y)
        theta = np.arctan2(X - radar_x, Y - radar_y)                     # radians
        az_angle = theta                # radians

        # Outgoing pulse ring
        pulse_distance = current_time * c
        pulse_width = pulse_s * c
        Z_total = np.zeros_like(R_arr)
        in_pulse_out = (R_arr > pulse_distance - pulse_width / 2.0) & (R_arr < pulse_distance + pulse_width / 2.0)
        if np.any(in_pulse_out):
            Z_total[in_pulse_out] = 1.0

        # Apply antenna pattern
        Z_total *= antenna_pattern(az_angle, beam_deg)

        # Target reflections (visual ring)
        for tx, ty, ta in targets:
            r_tx = np.hypot(X - tx, Y - ty)
            r_radar_to_target = np.hypot(radar_x - tx, radar_y - ty)
            targ_az = np.arctan2(tx - radar_x, ty - radar_y)
            ant_amp = antenna_pattern(targ_az, beam_deg)

            if pulse_distance >= r_radar_to_target:
                leftover = pulse_distance - r_radar_to_target
                in_pulse_t = (r_tx > leftover - pulse_width/2.0) & (r_tx < leftover + pulse_width/2.0)
                if np.any(in_pulse_t):
                    Z_total[in_pulse_t] += ant_amp * ta

        # Update left image & artists
        im_wave.set_data(Z_total)
        # sc_radar.set_offsets(np.c_[[radar_x], [radar_y]])

        # Update targets scatter
        scatter_tg.set_offsets(np.c_[[t[0] for t in targets], [t[1] for t in targets]])

        # Build/update SAR heatmap (recompute rows up to last *completed* position)
        t_axis, range_axis = make_range_axis(pulse_s)
        echo_data = np.zeros((num_positions, t_axis.size), dtype=float)
        for p in range(min(pos_idx, num_positions)):  # exclude current in-progress row
            echo_data[p, :] = compute_row_echo(p, beam_deg, pulse_s, targets, t_axis)

        if im_sar is None:
            im_sar = ax_sar.imshow(
                echo_data, aspect='auto', cmap=cmap_dd.value,
                extent=sar_extent(range_axis), origin='upper'
            )
            ax_sar.set_title("Radar Echo Data")
            ax_sar.set_xlabel("Range (m)")
            ax_sar.set_ylabel("Radar X position (m)")
            sar_cbar = fig.colorbar(im_sar, ax=ax_sar, label='Echo Amplitude')  # <-- no plt.*
        else:
            im_sar.set_data(echo_data)
            im_sar.set_extent(sar_extent(range_axis))

        # Keep color scale consistent
        im_sar.set_clim(0, sum([t[2] for t in targets]))

        antenna_size = wavelength / np.radians(beam_deg)
        nb_elements = int(antenna_size / (wavelength/2))
        element_x = np.linspace(-antenna_size/2, antenna_size/2, nb_elements)
        element_y = np.zeros_like(element_x)
        line_aperture.set_data(element_x + radar_x, element_y)

        # Readout text
        readout.value = (
            f"pos={pos_idx+1}/{num_positions}, frame={frame_idx+1}/{total_frames}, azimuth position={radar_x_positions[pos_idx]:.1f} m | "
            f"beam={beam_deg:.1f}°, pulse={pulse_s:.2e}s | "
            f"T1=({targets[0][0]:.1f},{targets[0][1]:.1f}), "
            f"T2=({targets[1][0]:.1f},{targets[1][1]:.1f})"
        )

        # Draw figure into output
        with out:
            clear_output(wait=True)
            display(fig)

    # -----------------------
    # Wiring
    # -----------------------
    def maybe_draw(change=None):
        if auto_update.value:
            draw(t_slider.value)

    def _ensure_outputs_dir():
        out_dir = os.path.join('outputs')
        os.makedirs(out_dir, exist_ok=True)
        return out_dir

    def on_reset(_):
        t1x.value, t1y.value = 30.0, 200.0
        t2x.value, t2y.value = -60.0, 250.0
        beam.value = beamwidth_default
        pulse.value = pulse_default
        t_slider.value = 0
        draw(t_slider.value)

    def on_save(_):
        out_dir = _ensure_outputs_dir()
        ts = datetime.now().strftime('%Y%m%d_%H%M%S')
        base = f'slar_{ts}'
        fig.savefig(os.path.join(out_dir, base + '.png'), dpi=120, bbox_inches='tight')

    # Time controls
    t_slider.observe(lambda ch: draw(ch["new"]), names='value')

    # Param controls
    for w in (t1x, t1y, t2x, t2y, beam, pulse):
        w.observe(maybe_draw, names='value')
    
    def _update_colormap(_):
        im_wave.set_cmap(cmap_dd.value)
        if im_sar is not None:
            im_sar.set_cmap(cmap_dd.value)
        if auto_update.value:
            draw(t_slider.value)
    cmap_dd.observe(_update_colormap, names='value')

    # Manual render button
    render_btn.on_click(lambda _: draw(t_slider.value))
    reset_btn.on_click(on_reset)
    save_btn.on_click(on_save)

    # Initial render
    draw(t_slider.value)

    return VBox([controls1, controls2, controls3, controls4, readout, out])

# Build and show (safe alongside other sims / Voila):
slar_app = build_slar_sim()
display(slar_app)

### 🧪 Try this
- Scrub Time to watch echo rows accumulate into a range–azimuth image.
- Reduce **Beamwidth**; notice sharper azimuth separation (due to narrower illuminated strip).
- Shorten **Pulse**; see better range separation in the echo rows.
- Slide targets azimuth (x) vs. range (y) and compare effects.

## 🔍 Exploring the Limits of Resolution

In both cases above, the radar scan produces a 2D **range–angle map** of the scene. The only difference lies in the coordinate system, which changes with the scanning strategy. But how well can we actually **distinguish** two nearby targets? Let’s take a closer look. 👀  

### 📏 Range Resolution  

- **What it depends on:** The **pulse length** ⏱ — the shorter the pulse, the better the range resolution.  

- **Why?** Imagine two people talking at the same time with very similar voices. If their words overlap, it becomes impossible to tell them apart. In radar, when two targets are too close in range, their echoes overlap in the same way — instead of two distinct peaks, we see **one merged blob** in the range profile.  

- **Test it yourself:** Place **both targets directly in front of the radar** (`x = 0`) and move them apart in the **y-direction** (range). At some point you’ll notice the echoes separate into two peaks. That minimum separation distance is the **range resolution**.  

**In theory:**  
We could always improve resolution by using a **shorter pulse** — less overlap, clearer separation.  

**In practice:**  
- A shorter pulse means **less energy** ⚡. Radar transmits close to its **peak power**, and we can’t simply crank that up. Energy is *power × duration* — so if the duration is shorter, the total energy is smaller.  
- Think of it like a **water hose** 🚰: if the flow rate (power) is fixed, the only way to get more water (energy) is to let it run longer. A shorter time = less water.  
- Less energy → weaker echoes → reduced detection range 🚫.  
- Short pulses also require **faster sampling** 🎛 in the receiver, otherwise an echo might slip in **between two samples** and be missed entirely.  
- But higher sampling speeds also admit **more noise**, which again reduces the maximum detection range.  

**Result:**  
It would seem we can’t have **both** high range resolution **and** long detection range at the same time. This trade-off is especially important for **spaceborne radars** 🚀, which must detect very faint echoes from hundreds of kilometers away.  

---

### 🎯 Azimuth Resolution  

- **What it depends on:** The **beamwidth** of the radar array 🌐.  
- **Test it:** Place **both targets at the same range** (same y) and separate them in the **x-direction**.  
- If they are too close, their echoes merge in **angle**, and we can’t tell them apart.  
- Narrower beams (smaller beamwidth) = **better angular resolution** ✅.  

**A quick reminder:** 💡  
Beamwidth is set by the **size of the antenna array** 📏. The more elements we add and the wider the array becomes, the more focused the interference pattern gets 🎯. That means a **sharper beam** 📡, and therefore **better azimuth resolution** ✅. When scanning with such an antenna, we only illuminate and receive echoes from a very **narrow slice of the scene** 🔍 at a time, which is what allows us to separate side-by-side targets.  

**The limitation:**  
- Resolution also depends on how far the targets are from the radar: objects that are **closer** are easier to separate than those that are far away.  
- Fundamentally, beamwidth is tied to **wavelength ÷ antenna diameter**.  
- From space, achieving < 1 m azimuth resolution with **3 cm microwaves** would require antennas **several kilometers long**! 😱  

This gives us the second fundamental challenge: with a **real aperture antenna**, azimuth resolution is always limited by beamwidth. To go beyond this limit, we’ll need a new idea. 💡

### 🚧 The Big Problem  

From range resolution, we learned that **short pulses** help us separate targets in distance — but they come at the cost of weaker echoes and reduced detection range. From azimuth resolution, we saw that **narrow beams** give sharper separation — but they require antennas so large they’re impractical from space.  

We’ve run into two **fundamental limits**:  
1. **Range resolution** needs to suffer if we want to see far. 👎
2. High **Azimuth resolution** demands antennas kilometers long. 😕 

So are we stuck with blurry radar images forever? Or is there a clever way around both problems? 🤔  
*(Spoiler: there is — and it’s called **Synthetic Aperture Radar**.)* 🚀  


## 📌 Summary

- **Range–angle images** form by stacking pulse echoes over **steered angles** or **flight-track positions**.
- **Beamwidth** controls angular resolution; **pulse length** controls range resolution.
- Moving-platform SLAR produces images by sweeping the beam over the scene via **motion instead of beam steering**.
